# Measuring Prompt Energy in LLMs

Research tool for analyzing prompt characteristics vs. energy consumption across LLM models.

In [ ]:
# Initialize dependencies and load environment
import pandas as pd, openai, os, json, time, requests
from dotenv import load_dotenv
from codecarbon import EmissionsTracker

load_dotenv()

In [ ]:
# Load prompts data and initialize processed status
data = pd.read_json("data/prompts.jsonl", lines=True)
if 'processed' not in data.columns:
    data['processed'] = 0
    data.to_json("data/prompts.jsonl", orient='records', lines=True)

In [ ]:
# Setup API clients for different models
load_dotenv(override=True)
clients, models_to_test = {}, []

if openai_key := os.getenv("OPENAI_API_KEY"):
    clients['openai'] = openai.OpenAI(api_key=openai_key)
    models_to_test.append("openai")

if groq_key := os.getenv("GROQ_API_KEY"):
    clients['llama'] = groq_key
    models_to_test.append("llama")

if mistral_key := os.getenv("MISTRAL_API_KEY"):
    clients['mistral'] = mistral_key
    models_to_test.append("mistral")


In [ ]:
# Measure energy consumption and performance metrics
def track_performance(model_name, prompt, api_call_func):
    tracker = EmissionsTracker(log_level="ERROR", save_to_file=False)
    tracker.start()
    start_time = time.time()
    response_data, time_to_first_token = api_call_func(prompt)
    duration = time.time() - start_time
    energy_consumed_wh = tracker.stop() * 1000
    
    usage = response_data.get("usage", {})
    prompt_tokens = usage.get("prompt_tokens", usage.get("input_tokens", 0))
    completion_tokens = usage.get("completion_tokens", usage.get("output_tokens", 0))
    total_tokens = usage.get("total_tokens", prompt_tokens + completion_tokens)
    content = response_data.get("choices", [{}])[0].get("message", {}).get("content", "")
    
    return {
        "prompt": prompt,
        "model": response_data.get("model", model_name),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "duration": round(duration, 2),
        "time_to_first_token": round(time_to_first_token, 3),
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "total_tokens": total_tokens,
        "tokens_per_second": round(total_tokens / duration, 1) if duration > 0 else 0,
        "energy_consumed_wh": round(energy_consumed_wh, 4),
        "response": content
    }


In [ ]:
# API call functions for different models
def call_openai(prompt):
    start_time = time.time()
    response = clients['openai'].chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}], 
        max_tokens=50, temperature=0.3
    )
    return {
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        "choices": [{"message": {"content": choice.message.content}} for choice in response.choices],
        "model": response.model
    }, time.time() - start_time

def call_llama(prompt):
    start_time = time.time()
    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={"Authorization": f"Bearer {clients['llama']}"},
        json={"model": "llama-3.1-8b-instant", "messages": [{"role": "user", "content": prompt}], 
              "max_tokens": 50, "temperature": 0.3}
    )
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code}")
    return response.json(), time.time() - start_time

def call_mistral(prompt):
    start_time = time.time()
    response = requests.post(
        "https://api.mistral.ai/v1/chat/completions",
        headers={"Authorization": f"Bearer {clients['mistral']}"},
        json={"model": "mistral-large-latest", "messages": [{"role": "user", "content": prompt}], 
              "max_tokens": 50, "temperature": 0.3}
    )
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code}")
    resp_json = response.json()
    if "error" in resp_json:
        raise Exception(f"API Error: {resp_json['error']}")
    return resp_json, time.time() - start_time


In [ ]:
# Process prompts and save results incrementally
def run_performance_tests(max_prompts=2):
    global data
    unprocessed = data[data['processed'] == 0].head(max_prompts)
    if len(unprocessed) == 0:
        return
    
    api_calls = {"openai": call_openai, "llama": call_llama, "mistral": call_mistral}
    
    for idx, row in unprocessed.iterrows():
        prompt_text = row['prompt_text']
        for model_name in models_to_test:
            print(f"Processing prompt {idx + 1} with {model_name}")
            result = track_performance(model_name, prompt_text, api_calls[model_name])
            with open("data/energy.jsonl", "a") as f:
                f.write(json.dumps(result) + "\n")
        data.loc[idx, 'processed'] = 1
        data.to_json("data/prompts.jsonl", orient='records', lines=True)

if models_to_test:
    run_performance_tests(max_prompts=2)


In [ ]:
# Reset processed status to reprocess all prompts
data['processed'] = 0
data.to_json("data/prompts.jsonl", orient='records', lines=True)
